In [1]:
import os
import sys
import pprint
import pandas as pd

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.ResultFitter import ResultFitter
import json

In [16]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.5,
    'seed' : 1,
    'subsample_size' : 0.25
}

loadData = LoadData(**load_args)

full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
dataGen = DataGenerator(loadData)
resultFitter = ResultFitter(loadData)

AttributeError: 'LoadData' object has no attribute 'getDatasets'

In [3]:
result_file_name = 'results_LSTM_NARROW_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
model = resultFitter.fit_from_csv_and_index(result_file_name, 0, 2, 10, use_tensorboard = False, 
                               use_liveplots = True, use_custom_callback = False)

NameError: name 'resultFitter' is not defined

In [7]:
def f1_score(precision, recall):
    f1 = 2*((precision*recall)/(precision + recall))
    return f1

def create_f1_list(precision_df, recall_df):
    f1 = []
    for i in range(len(precision_df)):
        f1.append(f1_score(precision_df.loc[i], recall_df.loc[i]))
    return f1

#result_file_name = 'results_InceptionTime_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
result_file_name = 'results_InceptionTime_NARROW_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
df = GridSearchResultProcessor().get_results_df_by_name(result_file_name, 2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df_f1 = df.copy()
df_f1.columns=df_f1.columns.str.strip()
all_train_precision = df_f1['train_precision']
all_train_recall = df_f1['train_recall']
all_val_precision = df_f1['val_precision']
all_val_recall = df_f1['val_recall']
f1_train = create_f1_list(all_train_precision, all_train_recall)
f1_val = create_f1_list(all_val_precision, all_val_recall)
df_f1['train_f1'] = f1_train
df_f1['val_f1'] = f1_val
df_f1

,batch_size,epochs,learning_rate,optimizer,bottleneck_size,kernel_size,module_activation,module_output_activation,nr_modules,num_filters,output_activation,shortcut_activation,use_bottleneck,use_residuals,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
0,128,100,0.00001,adam,40,20,relu,sigmoid,1,30,sigmoid,softmax,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.219389,0.980141,0.934600,0.919862,0.336151,0.944374,0.858780,0.836694,0.927173,0.847593
2,128,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.116027,0.992175,0.974580,0.969246,0.336744,0.946210,0.871695,0.855463,0.971906,0.863502
3,512,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.204701,0.998214,0.981919,0.974427,0.391749,0.954116,0.841346,0.799593,0.978158,0.819938
4,768,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.140784,0.998701,0.996893,0.996604,0.408263,0.897812,0.840169,0.856371,0.996748,0.848193
5,1024,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.161831,0.998883,0.998461,0.998794,0.411179,0.872325,0.835449,0.880678,0.998627,0.857467
6,256,100,0.00100,adam,26,20,linear,tanh,6,40,sigmoid,softmax,False,True,0.641398,0.662050,0.662021,0.999911,0.640544,0.664213,0.664213,1.000000,0.796617,0.798231
7,256,100,0.00100,adam,26,20,relu,tanh,6,40,sigmoid,softmax,False,True,0.638258,0.661961,0.661961,0.999911,0.637392,0.664213,0.664213,1.000000,0.796574,0.798231
8,256,100,0.00100,adam,26,20,softmax,tanh,6,40,sigmoid,softmax,False,True,0.146560,0.990198,0.977509,0.975676,0.362454,0.924091,0.857873,0.856367,0.976592,0.857119
9,256,100,0.00100,adam,26,20,tanh,tanh,6,40,sigmoid,softmax,False,True,0.106805,0.995150,0.984730,0.981710,0.349780,0.935071,0.867338,0.859986,0.983218,0.863646


In [8]:
sorted_f1_df = df_f1.sort_values(by=['val_f1', 'train_f1'], axis = 0, ascending=False)
sorted_f1_df

,batch_size,epochs,learning_rate,optimizer,bottleneck_size,kernel_size,module_activation,module_output_activation,nr_modules,num_filters,output_activation,shortcut_activation,use_bottleneck,use_residuals,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
14,256,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,False,0.043317,0.984956,0.985322,0.992976,0.473966,0.887029,0.863882,0.911106,0.989134,0.886866
15,256,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,False,0.080202,0.958806,0.968424,0.995036,0.504267,0.850311,0.844201,0.928975,0.981550,0.884562
35,256,120,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.139707,0.992077,0.984819,0.984939,0.330054,0.926686,0.879657,0.889165,0.984879,0.884386
38,256,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,tanh,False,True,0.066773,0.972034,0.972952,0.987517,0.499487,0.880289,0.856971,0.908166,0.980180,0.881826
17,256,100,0.00100,adam,30,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.098025,0.993001,0.988370,0.989405,0.338380,0.924805,0.875601,0.884641,0.988887,0.880098
43,256,100,0.00100,adam,26,40,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.215893,0.982650,0.974639,0.979008,0.365554,0.904642,0.864934,0.890522,0.976818,0.877542
36,256,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,relu,False,True,0.054576,0.994759,0.981001,0.976424,0.429861,0.940746,0.878906,0.872653,0.978707,0.875769
18,256,100,0.00100,adam,26,20,sigmoid,tanh,6,30,sigmoid,softmax,False,True,0.100352,0.993889,0.988696,0.989001,0.347901,0.920947,0.870192,0.880118,0.988848,0.875127
10,256,100,0.00100,adam,26,20,sigmoid,relu,6,40,sigmoid,softmax,False,True,0.202694,0.985524,0.961529,0.955909,0.332700,0.939912,0.876953,0.870391,0.958711,0.873660
39,256,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,True,True,0.265097,0.953312,0.920455,0.925086,0.352032,0.911833,0.864934,0.881927,0.922765,0.873348


In [9]:
sorted_f1_df = df_f1.sort_values(by=['val_precision', 'train_precision'], axis = 0, ascending=False)
sorted_f1_df

,batch_size,epochs,learning_rate,optimizer,bottleneck_size,kernel_size,module_activation,module_output_activation,nr_modules,num_filters,output_activation,shortcut_activation,use_bottleneck,use_residuals,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
35,256,120,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.139707,0.992077,0.984819,0.984939,0.330054,0.926686,0.879657,0.889165,0.984879,0.884386
36,256,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,relu,False,True,0.054576,0.994759,0.981001,0.976424,0.429861,0.940746,0.878906,0.872653,0.978707,0.875769
31,256,100,0.00100,adam,26,20,sigmoid,tanh,12,40,sigmoid,softmax,False,True,0.134154,0.991666,0.973130,0.967562,0.326458,0.956940,0.877855,0.854558,0.970338,0.866050
10,256,100,0.00100,adam,26,20,sigmoid,relu,6,40,sigmoid,softmax,False,True,0.202694,0.985524,0.961529,0.955909,0.332700,0.939912,0.876953,0.870391,0.958711,0.873660
17,256,100,0.00100,adam,30,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.098025,0.993001,0.988370,0.989405,0.338380,0.924805,0.875601,0.884641,0.988887,0.880098
22,256,100,0.00100,adam,26,20,sigmoid,tanh,6,44,sigmoid,softmax,False,True,0.138321,0.992075,0.981179,0.979389,0.335990,0.934983,0.874549,0.871748,0.980283,0.873147
13,256,100,0.00100,adam,26,20,sigmoid,softmax,6,40,sigmoid,softmax,False,True,0.188374,0.986761,0.958156,0.949535,0.333124,0.938815,0.874549,0.867677,0.953826,0.871100
30,256,100,0.00100,adam,26,20,sigmoid,tanh,9,40,sigmoid,softmax,False,True,0.131727,0.992725,0.979403,0.976038,0.340979,0.936113,0.873197,0.868356,0.977718,0.870770
2,128,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.116027,0.992175,0.974580,0.969246,0.336744,0.946210,0.871695,0.855463,0.971906,0.863502
45,256,100,0.00100,rmsprop,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.149205,0.994648,0.983162,0.979850,0.341941,0.938223,0.870793,0.862248,0.981503,0.866500


In [11]:
sorted_f1_df = df_f1.sort_values(by=['val_loss', 'train_loss'], axis = 0, ascending=True)
sorted_f1_df

,batch_size,epochs,learning_rate,optimizer,bottleneck_size,kernel_size,module_activation,module_output_activation,nr_modules,num_filters,output_activation,shortcut_activation,use_bottleneck,use_residuals,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall,train_f1,val_f1
31,256,100,0.00100,adam,26,20,sigmoid,tanh,12,40,sigmoid,softmax,False,True,0.134154,0.991666,0.973130,0.967562,0.326458,0.956940,0.877855,0.854558,0.970338,0.866050
35,256,120,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.139707,0.992077,0.984819,0.984939,0.330054,0.926686,0.879657,0.889165,0.984879,0.884386
10,256,100,0.00100,adam,26,20,sigmoid,relu,6,40,sigmoid,softmax,False,True,0.202694,0.985524,0.961529,0.955909,0.332700,0.939912,0.876953,0.870391,0.958711,0.873660
13,256,100,0.00100,adam,26,20,sigmoid,softmax,6,40,sigmoid,softmax,False,True,0.188374,0.986761,0.958156,0.949535,0.333124,0.938815,0.874549,0.867677,0.953826,0.871100
22,256,100,0.00100,adam,26,20,sigmoid,tanh,6,44,sigmoid,softmax,False,True,0.138321,0.992075,0.981179,0.979389,0.335990,0.934983,0.874549,0.871748,0.980283,0.873147
1,64,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.219389,0.980141,0.934600,0.919862,0.336151,0.944374,0.858780,0.836694,0.927173,0.847593
2,128,100,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.116027,0.992175,0.974580,0.969246,0.336744,0.946210,0.871695,0.855463,0.971906,0.863502
17,256,100,0.00100,adam,30,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.098025,0.993001,0.988370,0.989405,0.338380,0.924805,0.875601,0.884641,0.988887,0.880098
34,256,110,0.00100,adam,26,20,sigmoid,tanh,6,40,sigmoid,softmax,False,True,0.129836,0.994894,0.986299,0.984353,0.340027,0.940139,0.868239,0.856141,0.985325,0.862148
30,256,100,0.00100,adam,26,20,sigmoid,tanh,9,40,sigmoid,softmax,False,True,0.131727,0.992725,0.979403,0.976038,0.340979,0.936113,0.873197,0.868356,0.977718,0.870770


In [12]:
df_f1.iloc[31]

batch_size                       256
epochs                           100
learning_rate                  0.001
optimizer                       adam
bottleneck_size                   26
kernel_size                       20
module_activation            sigmoid
module_output_activation        tanh
nr_modules                        12
num_filters                       40
output_activation            sigmoid
shortcut_activation          softmax
use_bottleneck                 False
use_residuals                   True
train_loss                  0.134154
train_accuracy              0.991666
train_precision              0.97313
train_recall                0.967562
val_loss                    0.326458
val_accuracy                 0.95694
val_precision               0.877855
val_recall                  0.854558
train_f1                    0.970338
val_f1                       0.86605
Name: 31, dtype: object

## Findings:
 - New optimal dropout_rate: 0.2 (train_acc: 98%, val_acc: 90%) Old 0.3 (train_acc: 96.8%, val_acc: 90.9%)
 - RELU and Sigmoid activation functions do not work with this model.
 - L1 regularization is counter productive (Less is more) so let l1 = 0.00
 - 32 start neurons appear to be optimal

In [161]:
df.iloc[40:]

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall
40,64,33,0.20000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,1.084770,0.782372,0.786742,0.937814,1.147155,0.755786,0.754972,0.923161
41,64,33,0.10000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,0.899954,0.950041,0.860890,0.833142,1.037570,0.891613,0.777936,0.751359
42,64,33,0.00100,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,1.925948,0.747005,0.749432,0.938575,1.981802,0.707387,0.688447,0.891990
43,64,33,0.00010,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,2.269218,0.669505,0.519271,0.535989,2.274575,0.654939,0.509470,0.526278
44,64,33,0.00001,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,2.271817,0.659082,0.502983,0.511301,2.273199,0.650253,0.501420,0.512142
45,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,30,0.806053,0.964770,0.904025,0.887121,1.025919,0.893439,0.792614,0.774918
46,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,32,0.841913,0.955342,0.908333,0.903420,1.085873,0.875353,0.787405,0.786517
47,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,36,0.910195,0.951840,0.911742,0.912532,1.132424,0.878223,0.798295,0.802465
48,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,40,0.988093,0.953485,0.908523,0.905616,1.212564,0.877102,0.792850,0.794128
49,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.3,17,5,0.0001,0.001,sigmoid,same,44,1.054804,0.920381,0.908570,0.943146,1.275719,0.836668,0.785038,0.833635


In [162]:
def f1_score(precision, recall):
    f1 = 2*((precision*recall)/(precision + recall))
    return f1

def create_f1_list(precision_df, recall_df):
    f1 = []
    for i in range(len(precision_df)):
        f1.append(f1_score(precision_df.loc[i], recall_df.loc[i]))
    return f1

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall,train_f1,val_f1
0,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32,0.843425,0.928650,0.862717,0.857514,0.970667,0.868316,0.775568,0.778812,0.891666,0.821132
1,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32,0.856955,0.949212,0.845839,0.809346,0.980064,0.903571,0.766572,0.724409,0.873717,0.804132
2,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.400,17,5,0.0001,0.001,sigmoid,same,32,0.832348,0.944839,0.879387,0.867630,1.003364,0.888981,0.783144,0.768074,0.904590,0.824117
3,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.200,17,5,0.0001,0.001,sigmoid,same,32,0.759288,0.980216,0.920956,0.898200,1.083491,0.900166,0.794034,0.774517,0.937417,0.832628
4,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.100,17,5,0.0001,0.001,sigmoid,same,32,0.765186,0.966016,0.935662,0.935273,1.217005,0.874209,0.786458,0.790981,0.950396,0.830515
5,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.010,17,5,0.0001,0.001,sigmoid,same,32,0.782132,0.983592,0.944435,0.931207,1.338903,0.889803,0.787405,0.774517,0.956683,0.828167
6,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.001,17,5,0.0001,0.001,sigmoid,same,32,0.793265,0.982209,0.929980,0.910209,1.372538,0.886949,0.779356,0.763780,0.944839,0.820769
7,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.000,17,5,0.0001,0.001,sigmoid,same,32,1.044652,0.971432,0.922752,0.909552,1.513449,0.885833,0.776989,0.760916,0.939474,0.818637
8,64,33,0.01000,2,adam,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,2.342889,0.822314,0.720338,0.734196,2.349560,0.814815,0.708807,0.724409,0.775760,0.766957
9,64,33,0.01000,2,rmsprop,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,1.584430,0.763097,0.711021,0.813967,1.595288,0.757818,0.705492,0.815319,0.787711,0.785517


In [170]:
sorted_f1_df

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons,train_loss,train_precision,train_accuracy,train_recall,val_loss,val_precision,val_accuracy,val_recall,train_f1,val_f1
19,64,37,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.710379,0.951974,0.920872,0.926614,0.963329,0.881151,0.815341,0.833214,0.939123,0.856512
10,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.796258,0.968050,0.905414,0.885676,1.000532,0.909462,0.813920,0.798139,0.925033,0.850172
25,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0000,0.001,sigmoid,same,32,0.456250,0.971378,0.921750,0.908007,0.697848,0.890650,0.805398,0.804581,0.938624,0.845431
11,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,16,0.559328,0.946777,0.902908,0.903436,0.761700,0.874040,0.799716,0.814603,0.924599,0.843275
13,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,64,1.184011,0.948904,0.915775,0.921833,1.459759,0.866363,0.795455,0.816750,0.935173,0.840825
47,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,36,0.910195,0.951840,0.911742,0.912532,1.132424,0.878223,0.798295,0.802465,0.931772,0.838636
36,32,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.719439,0.938639,0.903933,0.914316,0.929952,0.864355,0.794508,0.812976,0.926318,0.837878
51,64,33,0.01000,2,sgd,tanh,"[1, 4]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.862765,0.942918,0.909186,0.918035,1.087199,0.861698,0.792614,0.812976,0.930310,0.836628
14,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,128,1.946189,0.955922,0.913227,0.910284,2.231662,0.885600,0.794981,0.792412,0.932545,0.836419
29,64,33,0.01000,2,sgd,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32,0.774593,0.961107,0.907754,0.896265,0.985839,0.893409,0.796402,0.785970,0.927554,0.836253


In [155]:
pd.to_numeric(df_f1['val_f1'], errors='coerce')

0     0.821132
1     0.804132
2     0.824117
3     0.832628
4     0.830515
5     0.828167
6     0.820769
7     0.818637
8     0.766957
9     0.785517
10    0.850172
11    0.843275
12    0.831845
13    0.840825
14    0.836419
15    0.809115
16    0.800293
17    0.790123
18    0.828972
19    0.856512
20    0.796238
21    0.796238
22    0.796238
23    0.796238
24    0.805288
25    0.845431
26    0.796238
27    0.796238
28    0.796238
29    0.836253
30   NaN      
31    0.761004
32    0.831517
33    0.784201
34    0.821750
35    0.806797
36    0.837878
37    0.833272
38    0.818396
39    0.789155
40    0.831131
41    0.815500
42    0.789035
43    0.583601
44    0.572993
45    0.829969
46    0.828561
47    0.838636
48    0.833555
49    0.835149
50    0.828306
51    0.836628
52    0.825959
53    0.832174
54    0.780612
55    0.775374
56    0.756599
57    0.712715
58    0.754318
59    0.806134
60    0.789657
61    0.795973
62    0.819888
63    0.761058
64   NaN      
Name: val_f1, dtype: floa

In [152]:
df_f1['val_f1'].isnull().values.any()

True

In [156]:
df_f1.columns

Index(['batch_size', 'epochs', 'learning_rate', 'num_layers', 'optimizer',
       'activation', 'decay_sequence', 'dropout_rate', 'filters',
       'kernel_size', 'l1_r', 'l2_r', 'output_layer_activation', 'padding',
       'start_neurons', 'train_loss', 'train_precision', 'train_accuracy',
       'train_recall', 'val_loss', 'val_precision', 'val_accuracy',
       'val_recall', 'train_f1', 'val_f1'],
      dtype='object')